<a href="https://colab.research.google.com/github/4en1x/diploma/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install gcc-4.8 g++-4.8 -y -qq;
!ln -s /usr/bin/gcc-4.8 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-4.8 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.8 60 --slave /usr/bin/g++ g++ /usr/bin/g++-4.8

38 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2019-03-01 09:48:52--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.189.146
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.189.146|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?cbTcLC-YAvnXkcJHzZnVUcWzU5hZ4wn-TcIdkwkss8JEYx6N8pvwTUn07zIIYmpEg39HfEYLnvLPCartc1TNWUNfpsrKv-8dzRAfZgtDDdllwft7vhjGVg3kxSGapknepwLyK2ROCNlyZ3v1lWI8NG9DTyPWvIEBpcGHsorvGUeNNLwUJ6PPxYHVtSKcPW73h7MFVc6U3s-shZ2nye3Y9BDq7A [following]
--2019-03-01 09:48:53--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?cbTcLC-YAvnXkcJHzZnVUc

In [0]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-0h9f_v5b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ydgzb7f9/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [0]:
!cd /usr/src
!ln -sfn linux-headers-2.6.35-28-generic linux

CODE (AT LEAST NOW)

In [0]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [0]:
%%cu
#include <stdio.h>

int main() {
	int deviceCount;
  cudaDeviceProp devProp;
    
  cudaGetDeviceCount(&deviceCount);
  printf("Found %d devices\n", deviceCount);
	return 0;
}


'Found 1 devices\n'

In [26]:
%%cu
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include "linux/kernel.h"
#include <iostream>
#include <ctime>
#include <iomanip>
#include <fstream>
#include <math.h>
#include <cmath>

using namespace std;

#define ITERATIONS 100
#define N 18

#define e 2.7182818
#define Dx 1
#define h ((float) Dx / N)
#define h2 powf(h, 2.0)

#define max(x, y) (((x) > (y)) ? (x) : (y))
#define min(x, y) (((x) < (y)) ? (x) : (y))


#define blockZSize

#define blockXSize 8
#define blockYSize 8

#define tileXSize 4
#define tileYSize 4

__device__
float f(int i, int j) {
	return 2 * powf(e, i * h + j * h);
}

float u_t(float x) {
	return powf(e, x);
}

float u_l(float x) {
	return powf(e, x);
}

float u_r(float x) {
	return powf(e, x + 1);
}

float u_d(float x) {
	return powf(e, x + 1);
}


__device__
float u(float* matrix, int i, int j) {
	return matrix[N * i + j];
}

__device__
void tile(float* matrix, int iGl, int jGl, bool isBlack) {
	int iBegin = iGl * blockXSize + threadIdx.x * tileXSize + 1;
	int jBegin = jGl * blockYSize + threadIdx.y * tileYSize + 1;

  //printf("iGl : %d, %d, %d, %d, %d, %d, %d \n", iGl, jGl, threadIdx.x, threadIdx.y, isBlack, iBegin, jBegin);

	for (size_t i = iBegin; i < iBegin + tileXSize; i++) {
		int offset = i % 2;
		if (isBlack) {
			offset = 1 - offset;
		}
		for (size_t j = jBegin + offset; j < jBegin + tileYSize; j += 2) {
      //printf("iGl : %d |", j);
			matrix[N * i + j] = 0.25 * (
          u(matrix, i + 1, j)
				+ u(matrix, i - 1, j)
				+ u(matrix, i, j + 1)
				+ u(matrix, i, j - 1) - h2 * f(i, j));
		}
	}
}

__global__
void calculateBlack(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, true);
}

__global__
void calculateRed(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, false);
}

float func(int i, int j) {
	return powf(e, i * h + j * h);
}

float* createMatrix(int row, int col) {
	float* matrix = new float[row * col];
	for(size_t i = 0; i < row; i++) {
		for(size_t j = 0; j < col; j++) {
			matrix[i * row + j] = 0;
		}
	}
    
  for(size_t i = 1; i < row - 1; i++) {
    matrix[0 * row + i] = u_t(i * h);
    matrix[(row - 1) * row + i] = u_d(i * h);
	}

  for(size_t j = 1; j < col - 1; j++) {
    matrix[j * col + 0] = u_l(j * h);
    matrix[j * col + row - 1] = u_r(j * h);
	}
    
	return matrix;
}

float calculateDifference(float* matrix1) {
	float tolerance = 0.0f;
	for (size_t i = 0; i < N * N; i++) {
    float fCurrent = func(i / N, i % N);
		tolerance += (matrix1[i] - fCurrent) * (matrix1[i] - fCurrent);
	}
  
	return tolerance;
}

int main() {
  ofstream myfile;
  myfile.open("example.txt");
	float* matrixHost = createMatrix(N, N);
  
/*
  for (size_t i = 0; i < N; i++) {
		for (size_t j = 0; j < N; j++) {
      myfile << matrixHost[i * N + j] << " ";
		}
    myfile << "\n";
	}
*/
  

	float* matrixDevice;
	
	int matrixSize = sizeof(float) * N * N;

	cudaMalloc((void**)&matrixDevice, matrixSize);
	cudaMemcpy(matrixDevice, matrixHost, matrixSize, cudaMemcpyHostToDevice);

	long startTime = clock();
	dim3 blocks((N - 2) / blockXSize, (N - 2) / blockYSize);
	dim3 threads(blockXSize / tileXSize, blockYSize / tileYSize);

	for (size_t i = 0; i < ITERATIONS; i++) {
		cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

		calculateBlack <<< blocks, threads >>>(matrixDevice);
		cudaDeviceSynchronize();
		calculateRed <<< blocks, threads >>>(matrixDevice);

		cudaPeekAtLastError();
		cudaDeviceSynchronize();

		cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

      
    cout << calculateDifference(matrixHost) << "\n";
      /*
    for (size_t i = 0; i < N; i++) {
		  for (size_t j = 0; j < N; j++) {
        myfile << matrixHost[i * N + j] << " ";
		  }
      myfile << "\n";
	 }
      */
     
	}
	std::cout << "Calculation take time: " << clock() - startTime << std::endl;
	cudaError_t cudaResult = cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

	if (cudaResult != cudaSuccess) {
		std::cout << "Cuda error: " << cudaResult << std::endl;
	}
  myfile.close();
	return 0;
}


'1859.85\n1584.34\n1398.53\n1255.73\n1139.29\n1040.99\n956.058\n881.462\n815.136\n755.609\n701.794\n652.863\n608.175\n567.217\n529.573\n494.899\n462.905\n433.342\n405.997\n380.681\n357.228\n335.488\n315.329\n296.629\n279.278\n263.177\n248.232\n234.361\n221.485\n209.533\n198.439\n188.141\n178.584\n169.715\n161.485\n153.849\n146.766\n140.197\n134.106\n128.459\n123.224\n118.374\n113.881\n109.72\n105.868\n102.303\n99.0042\n95.9538\n93.1338\n90.528\n88.1213\n85.8994\n83.8494\n81.9588\n80.2164\n78.6115\n77.1342\n75.7753\n74.5264\n73.3794\n72.3269\n71.3621\n70.4784\n69.67\n68.9314\n68.2572\n67.6428\n67.0836\n66.5755\n66.1147\n65.6975\n65.3207\n64.981\n64.6757\n64.4021\n64.1576\n63.94\n63.7472\n63.577\n63.4278\n63.2978\n63.1854\n63.0891\n63.0076\n62.9395\n62.8839\n62.8395\n62.8053\n62.7805\n62.7641\n62.7554\n62.7537\n62.7583\n62.7685\n62.7837\n62.8036\n62.8274\n62.8549\n62.8856\n62.919\nCalculation take time: 9903\n'

In [0]:
import os

f = open("example.txt", 'r')
i = 0

content = f.readlines()
for line in content:
  if i % 8 == 0:
    print('\n')
  line = line.split(' ')
  for el in line:
    print(el.rjust(12), end='')
  i += 1
